In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn.tree import export_graphviz
from IPython.display import Image  
from pydotplus import graph_from_dot_data

In [15]:
df = pd.read_csv("/Users/schlinkertc/Flatiron/projects/classification/files/data.csv",index_col='cik')
df['manufacturing'] = df['sic'].map(lambda x: 3000<x<4000)
df.drop(columns='sic',inplace=True)

logistic

In [28]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)


//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [29]:
dictionary = dict(zip(list(X_train.columns), list(logreg.coef_[0])))

In [30]:
dictionary

{'Income/OperationsCash': 2.960368115953984e-16,
 'Income/FinancingCash': -1.0197287023880328e-15,
 'Income/Investing': 7.715779442994331e-16,
 'Assets': 1.0600421436169843e-10,
 'LiabilitiesAndStockholdersEquity': -2.0710441930832194e-10,
 'StockholdersEquity': 1.3467703315613482e-11,
 'NetIncomeLoss': 8.370620079286459e-11,
 'NetCashProvidedByUsedInOperatingActivities': 3.145891338197483e-11,
 'NetCashProvidedByUsedInFinancingActivities': -5.657349168090435e-10,
 'NetCashProvidedByUsedInInvestingActivities': 2.8996726043576434e-12,
 'debt_ratio': -5.320059928503157e-15}

In [31]:
y_pred_class = logreg.predict(X_test)

In [32]:
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

0.7795275590551181


In [33]:
y_test.value_counts()

False    1220
True      304
Name: manufacturing, dtype: int64

In [34]:
print(y_test.mean(),1-y_test.mean())

0.1994750656167979 0.8005249343832022


## SGD

In [16]:
scaler = StandardScaler()

X=df.drop(columns=['adsh','manufacturing'])
y=df['manufacturing']

X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=.2,random_state=10)

In [18]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state=12)
sgd_clf.fit(X_train,y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=12, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [21]:
sgd_clf.predict(X_test)

array([False, False,  True, ...,  True, False, False])

In [26]:
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf,X_train, y_train,cv=4,scoring='accuracy')

array([0.79213115, 0.75393701, 0.77281681, 0.65988181])

### Base Estimator

In [36]:
from sklearn.base import BaseEstimator

In [38]:
class Not_class(BaseEstimator):
    def fit(self,X,y=None):
        pass
    def predict(self,X):
        return np.zeros((len(X),1), dtype=bool)
        

In [39]:
not_manufacturing = Not_class()
cross_val_score(not_manufacturing,X_train,y_train,cv=3,scoring='accuracy')

array([0.79379921, 0.80068898, 0.82028557])

### Confusion Matrix

In [41]:
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(sgd_clf, X_train, y_train, cv=3)

In [42]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train,y_train_pred)

array([[3623, 1283],
       [ 789,  400]])

## Decision Tree

In [9]:
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)

y_preds = clf.predict(X_test)
accuracy_score(y_test,y_preds)

0.734251968503937

In [27]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state=0)
rfc

rfc.fit(X_train,y_train)
rfc_preds = rfc.predict(X_test)
accuracy_score(y_test,rfc_preds)

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.797244094488189

In [11]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators':[36,37]},
    
]

# grid_search = GridSearchCV(rfc,param_grid,cv=5,
#                           scoring='accuracy',
#                           return_train_score=True)